# プロジェクション層

プロジェクション（Projection）層は、DNNの中間においてデータ変換を行う層です。フロントエンド層と同じような構造が使われる場合もあります。

本セクションはチュートリアル[Deep Learning 101 for Audio-based MIR](https://geoffroypeeters.github.io/deeplearning-101-audiomir_book/bricks_projection.html)の内容に基づいています。

## 全連結層

全連結（Fully-Connected）層は、Multi-Layer-Perceptron（MLP）やFeed-Forward（FF）構造の核心となる部品です。

`nn.Linear`のほかに、カーネルサイズ１の畳み込み層で実装する方法もあります。


In [1]:
import torch

input_tensor = torch.randn(8, 128)
fc_layer = torch.nn.Linear(in_features=128, out_features=256)
fc_conv_layer = torch.nn.Conv1d(in_channels=128, out_channels=256, kernel_size=1)

output_tensor = fc_layer(input_tensor)
output_tensor_conv = fc_conv_layer(input_tensor.unsqueeze(2))

print("Shape of output tensor from Linear layer: ", output_tensor.shape)
print("Shape of output tensor from Conv1d layer: ", output_tensor_conv.shape)

Shape of output tensor from Linear layer:  torch.Size([8, 256])
Shape of output tensor from Conv1d layer:  torch.Size([8, 256, 1])


## ResNet

「**残差連結（residual/skip connection）**」を持つネットワーク構造。「入力から出力への変換」ではなく「入力と出力の差分」を学習する構造にすることで、層数が大きいニューラルネットの学習が劇的に容易になります。実装も極めて容易（入・出力を足し合わせるだけ）で、今ではTransformer始め多くの深層ニューラルネットに採用されている定番構造です。

In [2]:
class ResidualBlock(torch.nn.Module):
    def __init__(self, channels, kernel_size=3, stride=1, padding=1):
        super().__init__()
        self.conv1 = torch.nn.Conv1d(channels, channels, kernel_size, stride, padding)
        self.bn1 = torch.nn.BatchNorm1d(channels)
        self.relu = torch.nn.ReLU()
        self.conv2 = torch.nn.Conv1d(channels, channels, kernel_size, stride, padding)
        self.bn2 = torch.nn.BatchNorm1d(channels)
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual
        out = self.relu(out)
        return out


residual_block = ResidualBlock(channels = 128)

input_tensor = torch.randn(8, 128, 256)
output_tensor = residual_block(input_tensor)
print("Shape of output tensor from Residual block: ", output_tensor.shape)

Shape of output tensor from Residual block:  torch.Size([8, 128, 256])
